In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Carga de archivos
df_clientes = pd.read_csv(r"data/raw/clientes.csv")
df_productos = pd.read_csv(r"data/raw/productos.csv")
df_ventas = pd.read_csv(r"data/raw/ventas.csv")
df_detalle = pd.read_csv(r"data/raw/detalle_ventas.csv")

In [14]:
# Limpieza: Clientes

df_clientes['fecha_alta'] = pd.to_datetime(df_clientes['fecha_alta'])
df_clientes = df_clientes.drop_duplicates(subset=['id_cliente'], keep='first')
df_clientes['email'] = df_clientes['email'].fillna('desconocido@mail.com')
df_clientes['ciudad'] = df_clientes['ciudad'].fillna('Desconocida')

print(df_clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1531 entries, 0 to 1530
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_cliente      1531 non-null   int64         
 1   nombre_cliente  1531 non-null   object        
 2   email           1531 non-null   object        
 3   ciudad          1531 non-null   object        
 4   fecha_alta      1531 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 59.9+ KB
None


In [15]:
# Limpieza: Productos

df_productos['precio_unitario'] = pd.to_numeric(df_productos['precio_unitario'], errors='coerce')
df_productos = df_productos.drop_duplicates(subset=['id_producto'], keep='first')
df_productos = df_productos.dropna(subset=['precio_unitario'])

print(df_productos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_producto      100 non-null    int64 
 1   nombre_producto  100 non-null    object
 2   categoria        100 non-null    object
 3   precio_unitario  100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ KB
None


In [16]:
# Limpieza: Ventas

df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])
df_ventas = df_ventas.drop_duplicates(subset=['id_venta'], keep='first')

# Eliminamos columnas redundantes identificadas en el README.md
df_ventas = df_ventas.dropna(subset=['id_cliente', 'fecha'])
df_ventas['id_cliente'] = df_ventas['id_cliente'].astype(int)

print(df_ventas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113882 entries, 0 to 113881
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id_venta        113882 non-null  int64         
 1   id_cliente      113882 non-null  int32         
 2   fecha           113882 non-null  datetime64[ns]
 3   nombre_cliente  113882 non-null  object        
 4   email           113882 non-null  object        
 5   medio_pago      113882 non-null  object        
dtypes: datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 4.8+ MB
None


In [17]:
# Limpieza: Detalle_Ventas

# Eliminamos columna redundante
df_detalle = df_detalle.drop(columns=['nombre_producto'])

df_detalle['cantidad'] = pd.to_numeric(df_detalle['cantidad'], errors='coerce')
df_detalle['precio_unitario'] = pd.to_numeric(df_detalle['precio_unitario'], errors='coerce')
df_detalle['importe'] = pd.to_numeric(df_detalle['importe'], errors='coerce')

# Una venta sin importe o cantidad no sirve
df_detalle = df_detalle.dropna(subset=['id_venta', 'id_producto', 'importe', 'cantidad'])

print(df_detalle.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279528 entries, 0 to 279527
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id_detalle       279528 non-null  int64  
 1   id_venta         279528 non-null  int64  
 2   id_producto      279528 non-null  int64  
 3   cantidad         279528 non-null  int64  
 4   precio_unitario  279528 non-null  float64
 5   importe          279528 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 12.8 MB
None


In [18]:
# Integridad Referencial

# 1. Ventas -> Clientes
clientes_validos = df_clientes['id_cliente'].unique()
ventas_invalidas = df_ventas[~df_ventas['id_cliente'].isin(clientes_validos)].shape[0]
print(f"Ventas con id_cliente 'huérfano': {ventas_invalidas}")
if ventas_invalidas > 0:
    df_ventas = df_ventas[df_ventas['id_cliente'].isin(clientes_validos)]

# 2. Detalle_Ventas -> Ventas
ventas_validas = df_ventas['id_venta'].unique()
detalles_invalidos = df_detalle[~df_detalle['id_venta'].isin(ventas_validas)].shape[0]
print(f"Detalles con id_venta 'huérfano': {detalles_invalidos}")
if detalles_invalidos > 0:
    df_detalle = df_detalle[df_detalle['id_venta'].isin(ventas_validas)]
    
print("Verificación de integridad completada. Datos huérfanos eliminados.")

Ventas con id_cliente 'huérfano': 0
Detalles con id_venta 'huérfano': 0
Verificación de integridad completada. Datos huérfanos eliminados.


In [19]:
df_detalle.to_csv(r"data/clean/detalle_ventas.csv", index=False)
df_clientes.to_csv(r"data/clean/clientes.csv", index=False)
df_productos.to_csv(r"data/clean/productos.csv", index=False)
df_ventas.to_csv(r"data/clean/ventas.csv", index=False)